## Reading large QTl files based off QTLStudy structure

In [1]:
using DLMReader
using Chain
using InMemoryDatasets
import Base.Threads.@threads
using StatsBase

export QTLStudy, GWAS, QTLStudy_from_pattern, GenVarInfo
include("../src/inputs.jl")
include("../src/mrStudyCis.jl")

mrStudyCis

In [2]:
GWAS_file_path = "/mnt/r6_data/GWAS_MRsuite_hg38/input_MRsuite_BMI_Yengo_hg38"
QTL_prots_path = "/home/samat33/CrickData_samat33/Decode_mapping_suite"
QTL_prots_tss_file = "/mnt/r6_data/Patrick/Input_Decode/decode_list_hg38"

tss_df = filereader(QTL_prots_tss_file, delimiter = '_', header=[:trait, :chr, :tss], eolwarn = false)

dropmissing!(tss_df)

Decode_pat = [TRAIT_NAME, "/", TRAIT_NAME, "_chromosome_", CHR, ".tsv"]
Decode_columns = Dict(1 => "id", 2 => CHR, 3 => POS, 4 => A_EFFECT, 5 => A_OTHER, 6 => BETA, 7 => SE, 8 => PVAL)
sep = '\t'

Decode = QTLStudy_from_pattern(QTL_prots_path,Decode_pat, tss_df.trait, tss_df.chr, tss_df.tss, Decode_columns, sep, true)

display(Decode.path_v)
display(Decode.traits_for_each_path)

┌ Warning: There are problems with parsing the input file at line 723 (observation 723) : 
│ Column 2 : chr::Int64 : Read from buffer ("Y")
│  the values are set as missing.
│ MORE DETAILS: 
│ trait::String = CDY1.7097.8, chr::Int64 = missing, tss::Int64 = 25622162
│ CDY1.7097.8_Y_25622162
└ @ DLMReader ~/.julia/packages/DLMReader/N5BLJ/src/util.jl:952
┌ Warning: There are problems with parsing the input file at line 1295 (observation 1295) : 
│ Column 2 : chr::Int64 : Read from buffer ("Y")
│  the values are set as missing.
│ MORE DETAILS: 
│ trait::String = EIF1AY.19224.5, chr::Int64 = missing, tss::Int64 = 20575776
│ EIF1AY.19224.5_Y_20575776
└ @ DLMReader ~/.julia/packages/DLMReader/N5BLJ/src/util.jl:952


4483-element Vector{String}:
 "/home/samat33/CrickData_samat33" ⋯ 34 bytes ⋯ "/A1BG.16561.9_chromosome_19.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 36 bytes ⋯ "A1CF.12423.38_chromosome_10.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 32 bytes ⋯ "2/A2M.3708.62_chromosome_12.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 38 bytes ⋯ "2ML1.15482.12_chromosome_12.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 38 bytes ⋯ "4GALT.8759.29_chromosome_22.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 37 bytes ⋯ "A4GNT.10835.25_chromosome_3.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 38 bytes ⋯ "AGAB.11926.23_chromosome_15.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 44 bytes ⋯ "HPPT.17320.19_chromosome_11.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 36 bytes ⋯ "ABCC6.8935.22_chromosome_16.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 41 bytes ⋯ "HD10.17370.186_chromosome_3.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 36 bytes ⋯ "ABHD12.7825.7_chromosome_20.tsv"
 "/home/samat33/CrickData_samat33" ⋯ 41 bytes ⋯ "HD14B.19325.21_chrom

4483-element Vector{Any}:
 "A1BG.16561.9"
 "A1CF.12423.38"
 "A2M.3708.62"
 "A2ML1.15482.12"
 "A4GALT.8759.29"
 "A4GNT.10835.25"
 "AAGAB.11926.23"
 "AASDHPPT.17320.19"
 "ABCC6.8935.22"
 "ABHD10.17370.186"
 "ABHD12.7825.7"
 "ABHD14B.19325.21"
 "ABL1.3341.33"
 ⋮
 "ZNF566.12795.2"
 "ZNF774.12760.34"
 "ZNF843.8321.27"
 "ZNRF3.10390.21"
 "ZNRF3.14122.132"
 "ZP4.7766.25"
 "ZPBP.8635.283"
 "ZPBP2.10710.23"
 "ZPLD1.5590.11"
 "ZWILCH.18302.204"
 "ZWINT.18286.3"
 "ZYX.13632.10"

Read each file : keep only relevant cols and smoosh it all together

In [3]:
Decode[1]

GWAS("/home/samat33/CrickData_samat33/Decode_mapping_suite/A1BG.16561.9/A1BG.16561.9_chromosome_19.tsv", Dict{Int64, Any}(5 => A_OTHER, 4 => A_EFFECT, 6 => BETA, 7 => SE, 2 => CHR, 8 => PVAL, 3 => POS, 1 => "id"), '\t', "A1BG.16561.9")

In [3]:
verify_and_simplify_columns(Decode)

-1

In [4]:
types, header = make_types_and_headers(Decode)

(Dict{Int64, DataType}(5 => String, 4 => String, 6 => Float64, 7 => Float64, 2 => Int8, 8 => Float64, 3 => Int32, 1 => String), [:x, :chr, :pos, :a_effect, :a_other, :β, :se, :pval])

Next : test speed lecture pour quelques fichiers seulement manuelement

In [12]:
file = Decode[1]
d = filereader(file.path, delimiter = file.separator, header = header, types = types, skipto=2, makeunique=true, eolwarn=false, threads=false)[:,collect(keys(file.columns))]

,a_other,a_effect,β,se,chr,pval,pos
,identity,identity,identity,identity,identity,identity,identity
,String?,String?,Float64?,Float64?,Int8?,Float64?,Int32?
1,G,A,0.0147,0.091362,19,0.872174,60980
2,G,A,0.2382,0.167335,19,0.154594,61397
3,G,A,-0.7483,0.474036,19,0.114434,61738
4,T,G,0.0601,0.058977,19,0.308183,62617
5,C,T,0.0877,0.051866,19,0.090859,62831
6,G,A,-0.0527,0.240393,19,0.826475,62904
7,A,G,-0.1166,0.081051,19,0.150265,62973
8,G,A,-0.009,0.083019,19,0.913671,63404


In [5]:
window = 500000
p_tresh = 5.0e-3
in_window(s::SubArray) = (s[1] == ref_dict[s[3]][1] && abs(s[2]-ref_dict[s[3]][2])≤window && s[4]<p_tresh)
ref_dict = Dict(zip(Decode.trait_v, zip(Decode.chr_v, Decode.tss_v)))

Dict{Any, Tuple{Any, Any}} with 4483 entries:
  "TAGLN.15640.54"  => (11, 117199370)
  "NDUFS4.10584.7"  => (5, 53560633)
  "ANGPT2.2602.2"   => (8, 6563409)
  "SMC3.14324.52"   => (10, 110567684)
  "SYT11.7089.42"   => (1, 155859567)
  "ABCC6.8935.22"   => (16, 16223522)
  "RAB35.13514.121" => (12, 120117502)
  "ITLN1.18830.1"   => (1, 160885180)
  "ITCH.12551.3"    => (20, 34363241)
  "NID2.16060.99"   => (14, 52069059)
  "TCN2.5584.21"    => (22, 30607003)
  "AKR1B1.16606.85" => (7, 134459284)
  "FGG.4989.7"      => (4, 154612967)
  "EIF4B.14675.20"  => (12, 53006282)
  "GP5.7185.29"     => (3, 194399266)
  "CD5.5596.75"     => (11, 61102489)
  "LCT.9017.58"     => (2, 135837184)
  "RBL2.13565.2"    => (16, 53433977)
  "EDN1.6495.14"    => (6, 12290361)
  "PPBP.16765.52"   => (4, 73988190)
  "TMCO5A.6907.17"  => (15, 37921939)
  "DIRAS1.17793.4"  => (19, 2721372)
  "EPHB3.9220.7"    => (3, 184561785)
  "GUSB.15562.24"   => (7, 65982215)
  "MGA.11587.5"     => (15, 41621134)
  ⋮     

In [6]:
data_vect = Vector{Dataset}(undef, 1200);

In [7]:
using Printf
using BenchmarkTools
using Folds
using BangBang

In [8]:

@time @threads for i in 1:1200
    @printf "\r%5i" i
    file = Decode[i]
    d = filereader(file.path, delimiter = file.separator, header = header, types = types, skipto=2, makeunique=true, eolwarn=false, threads = false)[:,collect(keys(file.columns))]
    d.trait = repeat([file.trait_name], nrow(d))
    filter!(d, [:chr, :pos, :trait, :pval], type = in_window, threads = false)
    data_vect[i] = d
end

  200433.295479 seconds (12.12 G allocations: 449.200 GiB, 25.61% gc time, 2.71% compilation time: 1% of which was recompilation)


**Mem taken 109 Gb. look workspace for temporary allocations + other optimisation**

In [9]:
df_final = Folds.reduce(vcat, data_vect, init = Dataset())

,a_other,a_effect,β,se,chr,pval,pos,trait
,identity,identity,identity,identity,identity,identity,identity,identity
,String?,String?,Float64?,Float64?,Int8?,Float64?,Int32?,String?
1,G,A,-0.0382,0.009274,19,3.801e-5,57853825,A1BG.16561.9
2,A,G,0.2982,0.104226,19,0.004222,57855116,A1BG.16561.9
3,G,A,0.1683,0.059636,19,0.004771,57856457,A1BG.16561.9
4,G,A,-0.0406,0.009287,19,1.232e-5,57858374,A1BG.16561.9
5,T,C,0.2696,0.085608,19,0.001637,57858571,A1BG.16561.9
6,G,A,-0.1667,0.024098,19,4.594e-12,57858894,A1BG.16561.9
7,C,A,-0.0395,0.009252,19,1.961e-5,57859289,A1BG.16561.9
8,T,G,-0.0385,0.00926,19,3.214e-5,57859359,A1BG.16561.9


**Idéee solution** : 
```
si nb_files $\geq$ nb_threads : @threads for_loop
else : for_loop with threads = true in filereader and filter
```